<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>
<h4>By: Alexander Stetzer</h4>

In [114]:
#Imports for dataframes and extra processes
import numpy as np
import pandas as pd

#Set columns and rows so that they all print out instead of cutoff view
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Used to get Foursquare Data
import requests 

#Webscraping 
from bs4 import BeautifulSoup

#KMeans Clustering import
from sklearn.cluster import KMeans 

#Colors for cluster map
import matplotlib.cm as cm
import matplotlib.colors as colors

#Folium used to create maps with markers
#!pip install folium
import folium

#Used to find lat, long of Toronto
#!pip install geopy
from geopy.geocoders import Nominatim

#JSON handling
import json
from pandas.io.json import json_normalize 

<h2>Part One:Toronto Neighborhoods and Postal Codes DataFrame</h2>

In [115]:
#url of the postal codes for toronto and processing of url to useable html
url = 'http://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url)
d = html.text

#Creation of the BeautifulSoup object
soup = BeautifulSoup(d, 'html5lib')


In [116]:
#Empty list used to gather all of the Postal Data
toronto_list = []
postal_table = soup.find('table')

#Loop to go through the table data and extract the Postal Codes, Boroughs, and Neighborhoods of Toronto
for item in postal_table.findAll('td'):
    code = {}
    if item.span.text == 'Not assigned': #used to remove all of the postal codes with no Borough assignment
        pass
    else:
        code['Postal Code'] = item.p.text[:3]
        code['Borough'] = item.span.text.split('(')[0]
        code['Neighborhood'] = item.span.text.split('(')[1].replace(' /', ',').replace(')','').strip(' ')
        toronto_list.append(code)
    
#Creation of the DataFrame from the list created using the previous loop
toronto_df = pd.DataFrame(toronto_list)
toronto_df['Borough']=toronto_df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

toronto_df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


<h2>Part Two: Latitude and Longitude of Neighborhoods</h2>

In [117]:
#Geospatial file used to get lat, long of each postal code 
file = 'Geospatial_Coordinates.csv'

#Reading the geospatial file
geo_df = pd.read_csv(file)
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [118]:
#Merging of the two dataframes but only with postal codes in the toronto data
torontoGeo_df = toronto_df.merge(geo_df, on = 'Postal Code', how = 'left')
torontoGeo_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<h2>Part Three: Clustering of Neighborhoods</h2>

In [119]:
#Address used to find the lat and long for toronto
address = 'Toronto, CA'

#Using geolocator to find the coordinates for the folium map
geolocator = Nominatim(user_agent='toronto')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

#Print out to show that the geolocator worked
print('Lat = {}, Long = {}'.format(latitude, longitude))

Lat = 43.6534817, Long = -79.3839347


<p>To make sure all of the neighborhood data appears to be working right, a folium map is created.</p>

In [120]:
#Creation of the folium Map for the Toronto map
toronto_map = folium.Map(location=[latitude,longitude], zoom_start = 10)

#Creation of the circle markers 
for lat, long, borough, neigh in zip(torontoGeo_df.Latitude, torontoGeo_df.Longitude, torontoGeo_df.Borough, torontoGeo_df.Neighborhood):
    label = '{}, {}'.format(neigh, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius = 5,
        popup = label,
        color = 'red',
        fill = True,
        fill_color = '3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(toronto_map)

toronto_map


<p>The next step is to get the venue data for each of the neighborhoods from the Foursquare API</p>

In [121]:
#Foursquare data
CLIENT_ID = 'B0Q23MY5HDPPYYD152WCVCELTQCORGX0WVA3E0PSNKBGEF21'
CLIENT_SECRET = 'HVDBDQNM4A50RDADN0BSHHFMZFEGPZT5ZROBBPAD3QII3Q0I'
VERSION = '20180605'
LIMIT = 100

In [122]:
#Function that creates a dataframe of all the venues from the foursquare api
def getVenues(borough, neighborhood, latitude, longitude, rad = 500):
    venues_list=[]
    for boro, neigh, lat, long in zip(borough, neighborhood, latitude, longitude):
        #print-out to check progress on the function
        print('{}, {}'.format(neigh, boro))
        
        #url creation for each neighborhood
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius=500&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, long, LIMIT)
        
        #results from Foursquare
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        #row creation for each venue and neighborhood
        venues_list.append([(
            boro,
            neigh,
            lat,
            long,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    
    #Creation of the final data frame    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 'Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', \
                                'Venue Latitude', 'Venue Longitude', 'Venue Category']
       
    return nearby_venues

<p>For the final part I have chosen to cluster using all of the neighborhoods. The next line finds all the venues for this region</p>

In [123]:
venues = getVenues(torontoGeo_df.Borough, torontoGeo_df.Neighborhood, torontoGeo_df.Latitude, torontoGeo_df.Longitude)    

Parkwoods, North York
Victoria Village, North York
Regent Park, Harbourfront, Downtown Toronto
Lawrence Manor, Lawrence Heights, North York
Ontario Provincial Government, Queen's Park
Islington Avenue, Etobicoke
Malvern, Rouge, Scarborough
Don MillsNorth, North York
Parkview Hill, Woodbine Gardens, East York
Garden District, Ryerson, Downtown Toronto
Glencairn, North York
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale, Etobicoke
Rouge Hill, Port Union, Highland Creek, Scarborough
Don MillsSouth, North York
Woodbine Heights, East York
St. James Town, Downtown Toronto
Humewood-Cedarvale, York
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood, Etobicoke
Guildwood, Morningside, West Hill, Scarborough
The Beaches, East Toronto
Berczy Park, Downtown Toronto
Caledonia-Fairbanks, York
Woburn, Scarborough
Leaside, East York
Central Bay Street, Downtown Toronto
Christie, Downtown Toronto
Cedarbrae, Scarborough
Hillcrest Village, North York
Bathurst Manor, Wils

In [124]:
#one hot encoding
toronto_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

#Neighborhood added back in
toronto_onehot['Neighborhood'] = venues['Neighborhood']

#way to fix teh neighborhood column as it was not being placed into the correct spot
non_neigh = toronto_onehot.iloc[:, toronto_onehot.columns != 'Neighborhood']

#Correction of the one hot encoding dataframe 
fixed_columns = ['Neighborhood'] + non_neigh.columns.tolist()
toronto_onehot = toronto_onehot[fixed_columns]

#Shape of the dataframe
toronto_onehot.shape

(2130, 272)

In [125]:
#Grouping of the data into neighborhoods 
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)
toronto_grouped.head()

(101, 272)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Knitting Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0

In [126]:
#Function to find the most common venue categories in each neighborhood 
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [127]:
#Number of top venues for the list and indicators of 1st, 2nd, and others
num_top_venues = 10
indic = ['st', 'nd', 'rd']

#Setup of the columns where neighborhood is the first and the most common values follow
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indic[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
#DataFrame of the sorted venues of toronto        
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted.Neighborhood = toronto_grouped.Neighborhood

#Filling of the final toronto venues
for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind,:], num_top_venues) 
    
toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Latin American Restaurant,Breakfast Spot,Lounge,Event Space,Falafel Restaurant,Ethiopian Restaurant,Escape Room,Electronics Store,Discount Store
1,"Alderwood, Long Branch",Pizza Place,Gym,Pharmacy,Playground,Coffee Shop,Athletics & Sports,Sandwich Place,Pub,Distribution Center,Dim Sum Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Grocery Store,Fried Chicken Joint,Bridal Shop,Sandwich Place,Diner,Restaurant,Intersection,Supermarket
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Restaurant,Coffee Shop,Greek Restaurant,Fast Food Restaurant,Indian Restaurant,Sushi Restaurant,Pub,Liquor Store


<p>The penultimate step is to do the KMeans Clustering and cluster the neighborhoods.</p>

In [128]:
# Number of clusters for the Kmeans clustering
kclusters = 8

#Dropping the neighborhood column as it has nothing to do with the machine learning 
toronto_grouped_cluster = toronto_grouped.drop('Neighborhood', 1)

#Creation of the KMeans object
kmeans = KMeans(n_clusters = kclusters, random_state=3).fit(toronto_grouped_cluster)

#Printout of all the labels for the cluster
kmeans.labels_

array([2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1,
       1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 2,
       6, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 4, 1, 2, 1, 1, 6, 1, 1, 2,
       2, 1, 1, 0, 2, 7, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 0, 1, 3, 2, 1,
       1, 1, 5, 1, 0, 2, 0, 1, 1, 1, 2, 0, 2])

In [133]:
#Can be uncommented if needed to be run more than once as it breaks due to an adding another 'Cluster Labels' column
#toronto_venues_sorted = toronto_venues_sorted.drop('Cluster Labels', 1)

#Instertion of the cluster labels into the sorted venues dataFrame
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#Merging of the Data together so that it can be shown together
torontoGeo = torontoGeo_df.reset_index()
toronto_merged = torontoGeo
toronto_merged = toronto_merged.merge(toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='inner')

#Printout showing the shape and first 5 rows of data
print(toronto_merged.shape)
toronto_merged.head()

(101, 17)


,index,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Food & Drink Shop,Fast Food Restaurant,Park,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
1,1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Breakfast Spot,Café,Pub,Theater,Performing Arts Venue,Shoe Store,Restaurant
3,3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Boutique,Vietnamese Restaurant,Miscellaneous Shop,Coffee Shop,Accessories Store,Eastern European Restaurant,Electronics Store,Dumpling Restaurant
4,4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Bank,Bar,Beer Bar,Spa,Smoothie Shop,Sandwich Place,Salad Place,Burrito Place


In [130]:
#Creation of the map with the colors as markers
map_clusters = folium.Map(location=[latitude, longitude], zoom_start = 10)

#Creation of the color arrays used to color the cluster markers
x = np.arange(kclusters)
ys = [(i + x + (i*x)**2) for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0,1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#Creation of the markers for the map coloring by cluster
markers_colors = []
for lat, long, poi, cluster in zip(toronto_merged.Latitude, toronto_merged.Longitude, toronto_merged.Neighborhood, toronto_merged['Cluster Labels'] ):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat, long],
        radius = 5,
        popup = label,
        color=rainbow[cluster-1],
        fill = True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
    
map_clusters